# Hopkins-UCSD vocabulary mapping collaboration

## Overview

This document pulls together working code that is specific to the OMOP mapping project. In particular, this notebook is the taken from the master notebook, and represents functionality for the mapping analysis (after mapping has been completed).

Most of the working functions have been moved to `Resources/custom_funcs.py`

## Code setup (run every time)

Imports:

In [2]:
import sys; sys.path.insert(1, 'Resources')
import pandas as pd
import numpy as np
import sklearn.metrics
import json
import os
from datetime import datetime
import sssom.io
import yaml
import plotly.graph_objects as go
from datamanagement import valuedef_update, get_eldef, get_valdef
from custom_funcs import *
%load_ext memory_profiler
%load_ext autoreload
%autoreload 2

Pulling core definitions:

In [3]:
df_eldef = get_eldef(); assert df_eldef.CUI.is_unique
df_valdef = get_valdef(); assert df_valdef.ID.is_unique

## What elements are included? (run as needed)

### Counts

#### Element counts

By exam area

In [4]:
df_elcount_byexamarea = pd.DataFrame(df_eldef.examArea.value_counts())\
    .reset_index().rename(columns={"index":"examArea", "examArea":"Element Counts"})

df_elcount_byexamarea.to_csv("Exports/DefinitionCounts/ElementsByExamArea.csv", index=False)
df_elcount_byexamarea

,examArea,Element Counts
0,Strabismus,80
1,Contact Lens Current Rx,69
2,Contact Lens Final Rx,52
3,Main Exam,52
4,Extraocular Movement,37
5,Visual Acuity,30
6,Manifest,29
7,Final Eyeglasses Rx,25
8,Wearing Rx,24
9,Cycloplegic,16


#### Prepopulated values

By exam area

In [5]:
df_valcount_byexamarea = pd.DataFrame(df_valdef.merge(df_eldef, on="CUI").examArea.value_counts())\
    .reset_index().rename(columns={"index":"examArea", "examArea":"Prepopulated Option Counts"})

df_valcount_byexamarea.to_csv("Exports/DefinitionCounts/PrepopulatedOptionsByExamArea.csv", index=False)
df_valcount_byexamarea

,examArea,Prepopulated Option Counts
0,Main Exam,375
1,Visual Acuity,27
2,Gonioscopy,22
3,Pupils,20
4,Dilation,12
5,Worth 4 Dot,10
6,Strabismus,8
7,Tonometry,6
8,Wearing Rx,4
9,Final Eyeglasses Rx,4


By data element

In [6]:
df_valcount_bydataelement = pd.DataFrame(df_valdef.CUI.value_counts()).rename(columns={"CUI":"Prepopulated Option Counts"}).merge(df_eldef, left_index=True, right_on="CUI")\
    [["CUI", "examArea", "dataElement", "Prepopulated Option Counts"]]\
        .drop(columns=["CUI"])

df_valcount_bydataelement.to_csv("Exports/DefinitionCounts/PrepopulatedOptionsByElement.csv", index=False)
df_valcount_bydataelement.reset_index(drop=True)

,examArea,dataElement,Prepopulated Option Counts
0,Main Exam,Right Eye Cornea,69
1,Main Exam,Right Eye Macula,59
2,Main Exam,Right Eye Conjunctiva,44
3,Main Exam,Right Eye Lids,37
4,Main Exam,Right Eye Periphery,33
5,Main Exam,External Right Eye,32
6,Visual Acuity,Method,25
7,Main Exam,Right Eye Lens,23
8,Main Exam,Right Eye Iris,18
9,Main Exam,Right Eye Vessels,17


## Analyse mappings (run as needed)

Create output directory, if doesn't already exist

In [7]:
save_out = True
outdir = create_outdir()

### Import mappings

Mapping document paths

In [8]:
df_el_sb_path = "Resources/Mappings/SB_ElementMapping.xlsx"
df_el_cc_path = "Resources/Mappings/CC_ElementMapping.xlsx"
df_el_consensus_path = "Resources/Mappings/CONS_ElementMapping.xlsx"
df_val_sb_path = "Resources/Mappings/SB_ValueMapping.xlsx"
df_val_cc_path = "Resources/Mappings/CC_ValueMapping.xlsx"
df_val_consensus_path = "Resources/Mappings/CONS_ValueMapping.xlsx"

Import as dataframes

In [9]:
# Element mappings
df_el_sb = pd.read_excel(df_el_sb_path)[["sourceCode", "equivalence", "conceptId", 'comment']]\
        .astype({"sourceCode":"string", "equivalence":"string", "conceptId":"Int64", "comment":"string"})
df_el_cc = pd.read_excel(df_el_cc_path)[["sourceCode", "equivalence", "conceptId", 'comment']]\
        .astype({"sourceCode":"string", "equivalence":"string", "conceptId":"Int64", "comment":"string"})
df_el_consensus = pd.read_excel(df_el_consensus_path)[["sourceCode", "equivalence", "conceptId", "comment"]]\
        .astype({"sourceCode":"string", "equivalence":"string", "conceptId":"Int64"})

# Value mappings
df_val_sb = pd.read_excel(df_val_sb_path)[["sourceCode", "equivalence", "conceptId", 'comment']]\
        .astype({"sourceCode":"Int64", "equivalence":"string", "conceptId":"Int64", "comment":"string"})
df_val_cc = pd.read_excel(df_val_cc_path)[["sourceCode", "equivalence", "conceptId", 'comment']]\
        .astype({"sourceCode":"Int64", "equivalence":"string", "conceptId":"Int64", "comment":"string"})
df_val_consensus = pd.read_excel(df_val_consensus_path)[["sourceCode", "equivalence", "conceptId", "comment"]]\
        .astype({"sourceCode":"Int64", "equivalence":"string", "conceptId":"Int64"})

# Assert that they're all the same shape
assert df_el_consensus.shape == df_el_sb.shape == df_el_cc.shape
assert df_val_consensus.shape == df_val_sb.shape == df_val_cc.shape

# The following section is to filter out exam areas that were excluded from the study
apply_filter = True

if apply_filter:
    (df_el_sb, df_el_cc, df_el_consensus) = custom_filter((df_el_sb, df_el_cc, df_el_consensus), 'element')
    (df_val_sb, df_val_cc, df_val_consensus) = custom_filter((df_val_sb, df_val_cc, df_val_consensus), 'value')

### Create get breakdown analysis values

In [10]:
analysis_version = 2
dict_analyse = {
    "SB elements":analyze_mapping(df_el_sb, analysis_version=analysis_version),
    "CC elements":analyze_mapping(df_el_cc, analysis_version=analysis_version),
    "SB values":analyze_mapping(df_val_sb, analysis_version=analysis_version),
    "CC values":analyze_mapping(df_val_cc, analysis_version=analysis_version),
    "CONS elements":analyze_mapping(df_el_consensus, analysis_version=analysis_version),
    "CONS values":analyze_mapping(df_val_consensus, analysis_version=analysis_version)
}

if save_out:
    with open(outdir + "/Analysis/filtered_values.json", 'w') as outfile:
        json.dump(dict_analyse, outfile, indent=4)

### Figures

##### Pie chart

In [8]:
# individual_maps = ["SB elements", "CC elements", "SB values", "CC values"]
# consensus_maps = ["CONS elements", "CONS values"]
# working_list = consensus_maps   #use this to select what's viewed

# for label_working in working_list:
#     dict_working = dict_analyse[label_working]
#     fig, ax = plt.subplots(ncols=3, figsize=[20,4])

#     labels, values = zip(*dict_working["equivalence"].items())
#     ax[0].pie(values, labels=labels);
#     ax[0].set_title("Equivalence");

#     labels, values = zip(*dict_working["unmapped"].items())
#     ax[1].pie(values, labels=labels);
#     ax[1].set_title("Flags for UNMAPPED");

#     labels, values = zip(*dict_working["wider"].items())
#     ax[2].pie(values, labels=labels);
#     ax[2].set_title("Flags for WIDER");

#     if save_out: plt.savefig(outdir + "/Analysis/" + label_working)

##### Sankey diagram

In [9]:
analyse_list = ["SB elements", "CC elements", "SB values", "CC values", "CONS elements", "CONS values"]

In [10]:
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    out = colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])
    return "rgb" + str(tuple(map(lambda x: round(x*255), out)))

In [11]:
colour_map = {'BASE': "#808080",
    'EQUAL': "#47dcf5",#            Stoled from INDIRECT "#ffb47b",
    'WIDER': "#33d6ab",
    'NARROWER': "#f27762",
    'UNMATCHED': "#d56f8c",
    'VALSMAPPED': "#ffacff",
    'NOMATCH': "#d56f8c",
    'INDIRECT': "#47dcf5",
    'SUBFIELD': "#000000",
    'LATERALITY': "#c5ed99",
    'CONCEPTMISSING': "#bc82fb",
    'CONCEPTMISSING&LATERALITY': "#fed575",
    'OTHER': "#ffacff" # give it the same colour as VALSMAPPED 
 }

In [35]:
default_colours = ["#808080", "#ffb47b", "#33d6ab", "#f27762", "#d56f8c", "#000000", "#d56f8c", "#c5ed99", "#47dcf5", "#ffacff", "#fed575", "#bc82fb"]
# default_colours_b = ["#808080", "#d56f8c","#33d6ab","#f27762","#ffb47b","#bc82fb","#c5ed99","#fed575","#ffacff","#47dcf5","#d56f8c"]

In [16]:
def __labelhierarchycategorize(label):
    if label in ["EQUAL", "WIDER", "NARROWER", "UNMATCHED"]:
        return 0.5
    else:
        return 1

def get_sankey_values_from_dictanalyse(mdict):
    dictkeymap = {
        "equivalence": "BASE",
        "unmapped":"UNMATCHED",
        "wider":"WIDER"
    }

    namemap_counter = 0
    namemap_dict = {}

    source = []
    target = []
    value = []
    labels = []
    xvalues = []
    yvalues = []
    node_colours = []
    link_colors = []

    iter = 0

    for dictkey in mdict.keys():
        if dictkeymap[dictkey] not in namemap_dict.keys():
            namemap_dict[dictkeymap[dictkey]] = namemap_counter
            namemap_counter += 1
            if dictkeymap[dictkey] == "BASE":
                labels.append("")#"COUNT: " + str(sum(mdict[dictkey].values())))
                xvalues.append(0)
                yvalues.append(0.0)
                node_colours.append(colour_map["BASE"])
        
        for label in mdict[dictkey]:
            if label not in namemap_dict.keys():
                namemap_dict[label] = namemap_counter
                namemap_counter += 1
                labels.append(label + ": " + str(mdict[dictkey][label]))
                xvalues.append(__labelhierarchycategorize(label))
                yvalues.append(0.2 - iter)
                node_colours.append(colour_map[label])
                iter += 0.02

            source.append(namemap_dict[dictkeymap[dictkey]])
            target.append(namemap_dict[label])
            value.append(mdict[dictkey][label])
            link_colors.append(lighten_color(colour_map[label]))
            
    return namemap_dict, source, target, value, labels, xvalues, yvalues, node_colours, link_colors

In [17]:
namemap_dict, source, target, value, labels, xvalues, yvalues, node_colours, link_colors = get_sankey_values_from_dictanalyse(\
    combine_analyse(dict_analyse["CONS elements"], dict_analyse["CONS values"]))

In [20]:
fig = go.Figure(data=[go.Sankey(
    arrangement = "snap",
    valueformat = ".0f",
    valuesuffix = "TWh",
    # Define nodes
    meta = dict(
      textposition="top center"
    ),
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label = list(map((lambda lab: ""), labels)),
      x = xvalues,
      y = yvalues,
      color = node_colours
    ),
    # Add links
    link = dict(
      source =  source,
      target =  target,
      value =  value,
      color = link_colors
))])

fig.update_layout(title_text="Analysis Sankey Diagram",
    font_size=15,
    width = 800,
    height=600)
    

### Create expanded flags spreadsheets to allow Excel analysis

In [21]:
if save_out:
    append_sourceel_origindex(append_vocabulary_id(append_sourceel_names(append_concept_names(expand_flags(df_el_sb)))))\
        [["orig_index", "examArea", "dataElement", "concept_name", "equivalence", "sourceCode", "conceptId", "vocabulary_id", "NOMATCH", "VALSMAPPED", "INDIRECT", "LATERALITY", "CONCEPTMISSING", "SUBFIELD"]]\
            .to_excel(outdir + "/Analysis/FlagsExpanded/elements_sb.xlsx", index=False)
    append_sourceel_origindex(append_vocabulary_id(append_sourceel_names(append_concept_names(expand_flags(df_el_cc)))))\
        [["orig_index", "examArea", "dataElement", "concept_name", "equivalence", "sourceCode", "conceptId", "vocabulary_id", "NOMATCH", "VALSMAPPED", "INDIRECT", "LATERALITY", "CONCEPTMISSING", "SUBFIELD"]]\
            .to_excel(outdir + "/Analysis/FlagsExpanded/elements_cc.xlsx", index=False)
    append_vocabulary_id(append_sourceval_names(append_concept_names(expand_flags(df_val_sb))))\
        [["examArea", "dataElement", "value", "concept_name", "equivalence", "sourceCode", "conceptId", "vocabulary_id", "NOMATCH", "LATERALITY", "CONCEPTMISSING", "SUBFIELD"]]\
            .to_excel(outdir + "/Analysis/FlagsExpanded/values_sb.xlsx", index=False)
    append_vocabulary_id(append_sourceval_names(append_concept_names(expand_flags(df_val_cc))))\
        [["examArea", "dataElement", "value", "concept_name", "equivalence", "sourceCode", "conceptId", "vocabulary_id", "NOMATCH", "LATERALITY", "CONCEPTMISSING", "SUBFIELD"]]\
            .to_excel(outdir + "/Analysis/FlagsExpanded/values_cc.xlsx", index=False)
    append_sourceel_origindex(append_vocabulary_id(append_sourceel_names(append_concept_names(expand_flags(df_el_consensus)))))\
        [["orig_index", "examArea", "dataElement", "concept_name", "equivalence", "sourceCode", "conceptId", "vocabulary_id", "NOMATCH", "VALSMAPPED", "INDIRECT", "LATERALITY", "CONCEPTMISSING", "SUBFIELD"]]\
            .to_excel(outdir + "/Analysis/FlagsExpanded/elements_consensus.xlsx", index=False)
    append_vocabulary_id(append_sourceval_names(append_concept_names(expand_flags(df_val_consensus))))\
        [["examArea", "dataElement", "value", "concept_name", "equivalence", "sourceCode", "conceptId", "vocabulary_id", "NOMATCH", "LATERALITY", "CONCEPTMISSING", "SUBFIELD"]]\
            .to_excel(outdir + "/Analysis/FlagsExpanded/values_consensus.xlsx", index=False)

# Verify above expansion
verify_list = ["elements_consensus", "values_consensus"]
for verify_string in verify_list:
    df_temp = pd.read_excel((outdir + "/Analysis/FlagsExpanded/" + verify_string + ".xlsx"))

    #Assert mutually exclusive "UNMATCHED" flags
    unmatched_flaglist = list({"NOMATCH", "VALSMAPPED", "INDIRECT", "SUBFIELD"} & set(df_temp.columns))
    assert (df_temp.loc[df_temp.equivalence == "UNMATCHED"][unmatched_flaglist].sum(axis=1) == 1).all()

    #Assert presence of "WIDER" flags
    wider_flaglist = list({"LATERALITY", "CONCEPTMISSING"} & set(df_temp.columns))
    assert (df_temp.loc[df_temp.equivalence == "WIDER"][wider_flaglist].sum(axis=1) > 0).all()

### Investigate specific subsets

In [1]:
df_analyse = df_el_consensus
equiv = "WIDER"
flag = "CONCEPTMISSING"

df_sliced = append_vocabulary_id(append_sourceel_names(append_concept_names(rows_by_equiv_and_flag(df_analyse, flag, equiv))))\
    [["sourceCode", "examArea", "dataElement", "equivalence", "conceptId", "concept_name", "vocabulary_id"]]

[print(index + ":\t" + str(value)) for index,value in df_sliced.vocabulary_id.value_counts().iteritems()]
print("Total:\t%d" % df_sliced.shape[0])
df_sliced.head(5)

### Breakdown by source vocabulary

In [23]:
append_vocabulary_id(append_sourceel_names(append_concept_names(rows_by_equiv_and_flag(df_analyse, flag, equiv))))\
    .vocabulary_id.value_counts()

SNOMED    36
LOINC      8
Name: vocabulary_id, dtype: int64

## Comparing screens (run as needed)

Create output directory, if doesn't already exist

In [3]:
save_out = True
outdir = create_outdir()

### Import mappings

Mapping document paths

In [4]:
df_el_wh_path = "Resources/Mappings/WH_ElementMapping.xlsx"
df_el_sb_path = "Resources/Mappings/SB_ElementMapping.xlsx"
df_el_cc_path = "Resources/Mappings/CC_ElementMapping.xlsx"
df_el_consensus_path = "Resources/Mappings/CONS_ElementMapping.xlsx"
df_val_wh_path = "Resources/Mappings/WH_ValueMapping.xlsx"
df_val_sb_path = "Resources/Mappings/SB_ValueMapping.xlsx"
df_val_cc_path = "Resources/Mappings/CC_ValueMapping.xlsx"
df_val_consensus_path = "Resources/Mappings/CONS_ValueMapping.xlsx"

Import mappings as dataframes

In [5]:
# Element mappings
df_el_wh = pd.read_excel(df_el_wh_path)[["sourceCode", "equivalence", "conceptId", 'comment']]\
        .astype({"sourceCode":"string", "equivalence":"string", "conceptId":"Int64", "comment":"string"})
df_el_sb = pd.read_excel(df_el_sb_path)[["sourceCode", "equivalence", "conceptId", 'comment']]\
        .astype({"sourceCode":"string", "equivalence":"string", "conceptId":"Int64", "comment":"string"})
df_el_cc = pd.read_excel(df_el_cc_path)[["sourceCode", "equivalence", "conceptId", 'comment']]\
        .astype({"sourceCode":"string", "equivalence":"string", "conceptId":"Int64", "comment":"string"})
df_el_consensus = pd.read_excel(df_el_consensus_path)[["sourceCode", "equivalence", "conceptId", "comment"]]\
        .astype({"sourceCode":"string", "equivalence":"string", "conceptId":"Int64"})

# Value mappings
df_val_wh = pd.read_excel(df_val_wh_path)[["sourceCode", "equivalence", "conceptId", 'comment']]\
        .astype({"sourceCode":"Int64", "equivalence":"string", "conceptId":"Int64", "comment":"string"})
df_val_sb = pd.read_excel(df_val_sb_path)[["sourceCode", "equivalence", "conceptId", 'comment']]\
        .astype({"sourceCode":"Int64", "equivalence":"string", "conceptId":"Int64", "comment":"string"})
df_val_cc = pd.read_excel(df_val_cc_path)[["sourceCode", "equivalence", "conceptId", 'comment']]\
        .astype({"sourceCode":"Int64", "equivalence":"string", "conceptId":"Int64", "comment":"string"})
df_val_consensus = pd.read_excel(df_val_consensus_path)[["sourceCode", "equivalence", "conceptId", "comment"]]\
        .astype({"sourceCode":"Int64", "equivalence":"string", "conceptId":"Int64"})

assert df_el_consensus.shape == df_el_sb.shape == df_el_cc.shape == df_el_wh.shape
assert df_val_consensus.shape == df_val_sb.shape == df_val_cc.shape == df_val_wh.shape

apply_filter = True

if apply_filter:
    (df_el_wh, df_el_sb, df_el_cc, df_el_consensus) = custom_filter((df_el_wh, df_el_sb, df_el_cc, df_el_consensus), 'element')
    (df_val_wh, df_val_sb, df_val_cc, df_val_consensus) = custom_filter((df_val_wh, df_val_sb, df_val_cc, df_val_consensus), 'value')

### Compare mapping sheet sizes (SB vs CC)

In [6]:
n_rows_sb = (df_el_sb.shape[0], df_val_sb.shape[0])
print("--Number of rows, Sally--\n\tElement Map: %d\n\tValue map: %d" % (n_rows_sb[0], n_rows_sb[1]))

--Number of rows, Sally--
	Element Map: 210
	Value map: 488


In [7]:
n_rows_cc = (df_el_cc.shape[0], df_val_cc.shape[0])
print("--Number of rows, Sally--\n\tElement Map: %d\n\tValue map: %d" % (n_rows_cc[0], n_rows_cc[1]))

--Number of rows, Sally--
	Element Map: 210
	Value map: 488


In [8]:
print("--Number of rows is equivalent--\n\tElement Map: %s\n\tValue map: %s" % \
    (n_rows_cc[0] == n_rows_sb[0], n_rows_sb[1] == n_rows_cc[1]))

--Number of rows is equivalent--
	Element Map: True
	Value map: True


### Row differences

#### Counts

Assert that all source code lists are the same (necessary for further valid processing)

In [6]:
verify_sourceCode_aligned(df_el_sb, df_el_cc)     # Should do this every time
verify_sourceCode_aligned(df_val_sb, df_val_cc)     # Should do this every time

sr_eldiff = (df_el_sb.drop(columns="comment") == df_el_cc.drop(columns="comment")).all(axis=1).value_counts()
sr_valdiff = (df_val_sb.drop(columns="comment") == df_val_cc.drop(columns="comment")).all(axis=1).value_counts()
mat_values = {"Elements":sr_eldiff, "Values":sr_valdiff}

df_compare = pd.DataFrame(mat_values).rename({True:"Matched", False:"Unmatched"})
df_compare["Both"] = df_compare.sum(axis=1)
df_compare["% Element Agreement"] = (df_compare.Elements / df_compare.Elements.sum())
df_compare["% Value Agreement"] = (df_compare.Values / df_compare.Values.sum())
df_compare["% Overall Agreement"] = (df_compare.sum(axis=1) / df_compare.sum(axis=1).sum())

df_compare

,Elements,Values,Both,% Element Agreement,% Value Agreement,% Overall Agreement
Matched,193,428,621,0.919048,0.877049,0.889697
Unmatched,17,60,77,0.080952,0.122951,0.110303


In [7]:
df_compare.to_excel(outdir + "/MapCompare/percent_agreement.xlsx", index=True)

#### Calculating Cohen's kappa

1) Equivalence label mapping

In [8]:
dict_ismapped = {"UNMATCHED":1, "NARROWER":2, "WIDER":3, "EQUAL":4}

In [9]:
sr_el_sb_ismapped = df_el_sb.equivalence.map(dict_ismapped)
sr_el_cc_ismapped = df_el_cc.equivalence.map(dict_ismapped)
sr_val_sb_ismapped = df_val_sb.equivalence.map(dict_ismapped)
sr_val_cc_ismapped = df_val_cc.equivalence.map(dict_ismapped)

el_kap = sklearn.metrics.cohen_kappa_score(sr_el_sb_ismapped, sr_el_cc_ismapped)
val_kap = sklearn.metrics.cohen_kappa_score(sr_val_sb_ismapped, sr_val_cc_ismapped)
total_kap = sklearn.metrics.cohen_kappa_score(pd.concat([sr_el_sb_ismapped, sr_val_sb_ismapped]),\
        pd.concat([sr_el_cc_ismapped, sr_val_cc_ismapped]))

print("Element kappa: %.3f" % el_kap)
print("Value kappa: %.3f" % val_kap)
print("Total kappa: %.3f" % total_kap)

Element kappa: 0.892
Value kappa: 0.851
Total kappa: 0.879


In [10]:
df_kappa = pd.DataFrame([["Elements", el_kap], ["Values", val_kap], ["Overall", total_kap]], columns=["Subset", "Equiv Kappa"])
df_kappa.to_excel(outdir + "/MapCompare/equivalence_kappa.xlsx", index=False)

2) Whether it's mapped as mappable or unmappable

In [11]:
dict_ismapped = {"UNMATCHED":False, "NARROWER":False, "WIDER":True, "EQUAL":True}

In [12]:
sr_el_sb_ismapped = df_el_sb.equivalence.map(dict_ismapped)
sr_el_cc_ismapped = df_el_cc.equivalence.map(dict_ismapped)
sr_val_sb_ismapped = df_val_sb.equivalence.map(dict_ismapped)
sr_val_cc_ismapped = df_val_cc.equivalence.map(dict_ismapped)

el_kap = sklearn.metrics.cohen_kappa_score(sr_el_sb_ismapped, sr_el_cc_ismapped)
val_kap = sklearn.metrics.cohen_kappa_score(sr_val_sb_ismapped, sr_val_cc_ismapped)
total_kap = sklearn.metrics.cohen_kappa_score(pd.concat([sr_el_sb_ismapped, sr_val_sb_ismapped]),\
        pd.concat([sr_el_cc_ismapped, sr_val_cc_ismapped]))

print("Element kappa: %.3f" % el_kap)
print("Value kappa: %.3f" % val_kap)
print("Total kappa: %.3f" % total_kap)

Element kappa: 0.940
Value kappa: 0.788
Total kappa: 0.903


In [13]:
df_kappa = pd.DataFrame([["Elements", el_kap], ["Values", val_kap], ["Overall", total_kap]], columns=["Subset", "Mapped/Unmapped Kappa"])
df_kappa.to_excel(outdir + "/MapCompare/mappable_kappa.xlsx", index=False)

#### Investigating discrepancies

In [14]:
verify_sourceCode_aligned(df_el_sb, df_el_cc)     # Should do this every time
verify_sourceCode_aligned(df_val_sb, df_val_cc)     # Should do this every time
index_eldiff = ~(df_el_sb.drop(columns="comment") == df_el_cc.drop(columns="comment")).all(axis=1)
index_valdiff = ~(df_val_sb.drop(columns="comment") == df_val_cc.drop(columns="comment")).all(axis=1)

Element comparisons

In [11]:
exportfile = True
verify_sourceCode_aligned(df_el_sb, df_el_cc)     # Should do this every time
verify_sourceCode_aligned(df_val_sb, df_val_cc)     # Should do this every time

temp_index_eldiff = ~(df_el_wh.drop(columns="comment") == df_el_cc.drop(columns="comment")).all(axis=1)

df_elcompare = append_concept_names(df_el_wh)\
    .join(append_concept_names(df_el_cc).drop(columns="sourceCode"), lsuffix="_sb", rsuffix="_cc")\
        .merge(df_eldef, left_on="sourceCode", right_on="CUI")\
            [["examArea", "dataElement", "sourceCode", "equivalence_sb", "conceptId_sb", "concept_name_sb", "equivalence_cc", "conceptId_cc", "concept_name_cc"]]\
                .loc[temp_index_eldiff]

if exportfile:
    df_elcompare.to_excel(outdir + "/MapCompare/CC_CONS_ElementDiscrepancies.xlsx", index=False)
    
df_elcompare.head(3)

In [12]:
exportfile = True
verify_sourceCode_aligned(df_el_sb, df_el_cc)     # Should do this every time
verify_sourceCode_aligned(df_val_sb, df_val_cc)     # Should do this every time

df_elcompare = append_concept_names(df_el_sb)\
    .join(append_concept_names(df_el_cc).drop(columns="sourceCode"), lsuffix="_sb", rsuffix="_cc")\
        .merge(df_eldef, left_on="sourceCode", right_on="CUI")\
            [["examArea", "dataElement", "sourceCode", "equivalence_sb", "conceptId_sb", "concept_name_sb", "equivalence_cc", "conceptId_cc", "concept_name_cc"]]\
                .loc[index_eldiff]

if exportfile:
    df_elcompare.to_excel(outdir + "/MapCompare/ElementDiscrepancies.xlsx", index=False)
    
df_elcompare.head(3)

Value comparisons

In [13]:
exportfile = True
verify_sourceCode_aligned(df_el_sb, df_el_cc)     # Should do this every time
verify_sourceCode_aligned(df_val_sb, df_val_cc)     # Should do this every time

df_valcompare = append_concept_names(df_val_sb)\
    .join(append_concept_names(df_val_cc).drop(columns="sourceCode"), lsuffix="_sb", rsuffix="_cc")\
        .merge(df_valdef, left_on="sourceCode", right_on="ID")\
            .merge(df_eldef, left_on="CUI", right_on="CUI")\
                [["examArea", "dataElement", "value", "sourceCode", "equivalence_sb", "conceptId_sb", "concept_name_sb", "equivalence_cc", "conceptId_cc", "concept_name_cc"]]\
                    .loc[index_valdiff]

if exportfile:
    df_valcompare.to_excel(outdir + "/MapCompare/ValueDiscrepancies.xlsx", index=False)

df_valcompare.head(3)

#### Flowchart

Create the DataFrame to hold our output

In [18]:
lab_list = ["wh", "sb", "cc", "cons"]
lab_dict = {"wh":[df_el_wh, df_val_wh], "sb":[df_el_sb, df_val_sb], "cc":[df_el_cc, df_val_cc], "cons":[df_el_consensus, df_val_consensus]}
iterables = [lab_list, ["conceptId", "equivalence", "both", "sum"]]

index = pd.MultiIndex.from_product(iterables, names=["grader", "diff_source"])
m_columns = pd.MultiIndex.from_product([lab_list, ["#", '%']], names=["grader", "diff_source"])
df_rowchange = pd.DataFrame(columns=m_columns, index=index).astype("Int64")\
    .astype({("wh","%"):"float", ("sb","%"):"float", ("cc","%"):"float", ("cons","%"):"float"})

Loop through to append the values into this DataFrame

In [19]:
n_els = df_el_consensus.shape[0] + df_val_consensus.shape[0]

for col_lab in lab_list:
    for row_lab in lab_list:
        # Load the dataframes to compare
        df_col_el = lab_dict[col_lab][0].drop(columns="comment").fillna(0)
        df_col_val = lab_dict[col_lab][1].drop(columns="comment").fillna(0)
        df_row_el = lab_dict[row_lab][0].drop(columns="comment").fillna(0)
        df_row_val = lab_dict[row_lab][1].drop(columns="comment").fillna(0)

        verify_sourceCode_aligned(df_col_el, df_row_el)
        verify_sourceCode_aligned(df_col_val, df_row_val)
        
        conc_diff = ((df_col_el.equivalence == df_row_el.equivalence) & ~(df_col_el.conceptId == df_row_el.conceptId)).sum()\
            + ((df_col_val.equivalence == df_row_val.equivalence) & ~(df_col_val.conceptId == df_row_val.conceptId)).sum()
        eq_diff = (~(df_col_el.equivalence == df_row_el.equivalence) & (df_col_el.conceptId == df_row_el.conceptId)).sum()\
            + (~(df_col_val.equivalence == df_row_val.equivalence) & (df_col_val.conceptId == df_row_val.conceptId)).sum()
        both_diff = (~(df_col_el.equivalence == df_row_el.equivalence) & ~(df_col_el.conceptId == df_row_el.conceptId)).sum()\
            + (~(df_col_val.equivalence == df_row_val.equivalence) & ~(df_col_val.conceptId == df_row_val.conceptId)).sum()
        sum_diff = (~((df_col_el == df_row_el).all(axis=1))).sum()\
            + (~((df_col_val == df_row_val).all(axis=1))).sum()

        assert sum_diff == eq_diff + conc_diff + both_diff
        assert df_col_el.shape[0] + df_col_val.shape[0] == n_els
        assert df_row_el.shape[0] + df_row_val.shape[0] == n_els

        df_rowchange.loc[(row_lab, "conceptId"), (col_lab, "#")] = conc_diff
        df_rowchange.loc[(row_lab, "equivalence"), (col_lab, "#")] = eq_diff
        df_rowchange.loc[(row_lab, "both"), (col_lab, "#")] = both_diff
        df_rowchange.loc[(row_lab, "sum"), (col_lab, "#")] = sum_diff
        df_rowchange.loc[(row_lab, "conceptId"), (col_lab, "%")] = conc_diff / n_els
        df_rowchange.loc[(row_lab, "equivalence"), (col_lab, "%")] = eq_diff / n_els
        df_rowchange.loc[(row_lab, "both"), (col_lab, "%")] = both_diff / n_els
        df_rowchange.loc[(row_lab, "sum"), (col_lab, "%")] = sum_diff / n_els

In [20]:
df_rowchange

grader               wh            sb             cc           cons          
diff_source           #         %   #         %    #         %    #         %
grader diff_source                                                           
wh     conceptId      0  0.000000  10  0.014327   24  0.034384   24  0.034384
       equivalence    0  0.000000  26  0.037249   26  0.037249   30  0.042980
       both           0  0.000000  34  0.048711   50  0.071633   53  0.075931
       sum            0  0.000000  70  0.100287  100  0.143266  107  0.153295
sb     conceptId     10  0.014327   0  0.000000   22  0.031519   19  0.027221
       equivalence   26  0.037249   0  0.000000   26  0.037249   15  0.021490
       both          34  0.048711   0  0.000000   29  0.041547   22  0.031519
       sum           70  0.100287   0  0.000000   77  0.110315   56  0.080229
cc     conceptId     24  0.034384  22  0.031519    0  0.000000   11  0.015759
       equivalence   26  0.037249  26  0.037249    0  0.000000   17  0.024355
       both          50  0.071633  29  0.041547    0  0.000000    6  0.008596
       sum          100  0.143266  77  0.110315    0  0.000000   34  0.048711
cons   conceptId     24  0.034384  19  0.027221   11  0.015759    0  0.000000
       equivalence   30  0.042980  15  0.021490   17  0.024355    0  0.000000
       both          53  0.075931  22  0.031519    6  0.008596    0  0.000000
       sum          107  0.153295  56  0.080229   34  0.048711    0  0.000000

In [21]:
df_rowchange.to_excel(outdir + "/MapCompare/rowchange.xlsx")